In [2]:
import pandas as pd


In [3]:
df = pd.read_pickle("cleaned_flights.pkl")

In [4]:
train_df=df[df['Price'].notna()]
test_df=df[df['Price'].isna()]

In [5]:
X_train=train_df.drop('Price',axis=1)
y_train=train_df.Price

In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10682 entries, 0 to 10682
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Total_Stops      10682 non-null  float64
 1   Additional_Info  10682 non-null  int64  
 2   Date             10682 non-null  int64  
 3   Month            10682 non-null  int64  
 4   Year             10682 non-null  int64  
 5   Arrival_hour     10682 non-null  int64  
 6   Arrival_minutes  10682 non-null  int64  
 7   Dept_hour        10682 non-null  int64  
 8   Dept_min         10682 non-null  int64  
 9   flight_duration  10682 non-null  int64  
 10  Airline_1        10682 non-null  bool   
 11  Airline_2        10682 non-null  bool   
 12  Airline_3        10682 non-null  bool   
 13  Airline_4        10682 non-null  bool   
 14  Airline_5        10682 non-null  bool   
 15  Airline_6        10682 non-null  bool   
 16  Airline_7        10682 non-null  bool   
 17  Airline_8        

In [7]:
y_train.info()

<class 'pandas.core.series.Series'>
Index: 10682 entries, 0 to 10682
Series name: Price
Non-Null Count  Dtype  
--------------  -----  
10682 non-null  float64
dtypes: float64(1)
memory usage: 166.9 KB


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2670 entries, 10683 to 13353
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Total_Stops      2670 non-null   float64
 1   Additional_Info  2670 non-null   int64  
 2   Price            0 non-null      float64
 3   Date             2670 non-null   int64  
 4   Month            2670 non-null   int64  
 5   Year             2670 non-null   int64  
 6   Arrival_hour     2670 non-null   int64  
 7   Arrival_minutes  2670 non-null   int64  
 8   Dept_hour        2670 non-null   int64  
 9   Dept_min         2670 non-null   int64  
 10  flight_duration  2670 non-null   int64  
 11  Airline_1        2670 non-null   bool   
 12  Airline_2        2670 non-null   bool   
 13  Airline_3        2670 non-null   bool   
 14  Airline_4        2670 non-null   bool   
 15  Airline_5        2670 non-null   bool   
 16  Airline_6        2670 non-null   bool   
 17  Airline_7     

In [9]:
test_df.drop('Price',axis=1,inplace=True)

/var/folders/zy/_ttp6dpd6y9930rzw7p1861h0000gn/T/ipykernel_21523/1543327150.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop('Price',axis=1,inplace=True)


In [10]:
from sklearn.model_selection import train_test_split

X_tr,X_tst,y_tr,y_tst=train_test_split(X_train,y_train,test_size=0.2,random_state=42)

In [11]:
!pip install tensorflow



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [12]:
X_tr.shape

(8545, 30)

In [13]:
from sklearn.preprocessing import StandardScaler
scaler_x=StandardScaler()
X_tr_scaled=scaler_x.fit_transform(X_tr)
X_tst_scaled=scaler_x.transform(X_tst)

In [14]:
scaler_y=StandardScaler()
y_tr_scaled=scaler_y.fit_transform(y_tr.values.reshape(-1,1))
y_tst_scaled=scaler_y.transform(y_tst.values.reshape(-1,1))

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [16]:
from tensorflow.keras import Input

model=Sequential([
    Input(shape=(X_tr_scaled.shape[1],)),
    Dense(64,activation='relu',name='l1'),
    Dense(32,activation='relu',name='l2'),
    Dense(1,name='l3'),
])
model.compile(optimizer='adam',loss='mse',metrics=['mae'])

In [17]:
X_cv_scaled,X_ts_scaled,y_cv_scaled,y_ts_scaled=train_test_split(X_tst_scaled,y_tst_scaled,test_size=0.5,random_state=42)

In [18]:
history=model.fit(
    X_tr_scaled,
    y_tr_scaled,
    validation_data=(X_cv_scaled,y_cv_scaled),
    epochs=50,
    batch_size=32,
)

Epoch 1/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 620us/step - loss: 0.4799 - mae: 0.4517 - val_loss: 0.2884 - val_mae: 0.3453
Epoch 2/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 386us/step - loss: 0.2714 - mae: 0.3286 - val_loss: 0.2925 - val_mae: 0.3281
Epoch 3/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 380us/step - loss: 0.2584 - mae: 0.3096 - val_loss: 0.3371 - val_mae: 0.3163
Epoch 4/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 379us/step - loss: 0.2942 - mae: 0.3022 - val_loss: 0.2890 - val_mae: 0.2962
Epoch 5/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 377us/step - loss: 0.2431 - mae: 0.2891 - val_loss: 0.2304 - val_mae: 0.2875
Epoch 6/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 378us/step - loss: 0.2179 - mae: 0.2820 - val_loss: 0.1567 - val_mae: 0.2700
Epoch 7/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 377us/step - loss: 0.2033 - mae: 0.2733 - val_loss: 0.1574 - val_mae: 0.2682
Epoch 8/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 379us/step - loss: 0.1979 - mae: 0.2721 - val_loss: 0.1472 - val_mae: 0.2635
Epoch 9/50
268/268 ━━━━━━━━━━━━━━━━━━━━ 

In [19]:
test_loss,test_mae=model.evaluate(X_ts_scaled,y_ts_scaled)
print("Test MSE:", test_loss)
print("Test MAE:", test_mae)

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step - loss: 0.1208 - mae: 0.2212
Test MSE: 0.12078198045492172
Test MAE: 0.22116787731647491


In [20]:
y_pred_scaled=model.predict(X_ts_scaled)
y_pred=scaler_y.inverse_transform(y_pred_scaled)
y_true = scaler_y.inverse_transform(y_ts_scaled)

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 631us/step


In [21]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae_real = mean_absolute_error(y_true, y_pred)
mse_real = mean_squared_error(y_true, y_pred)
print("MAE in ₹:", mae_real)
print("MSE in ₹^2:", mse_real)


MAE in ₹: 1020.4018705419785
MSE in ₹^2: 2570990.108060454


In [22]:
import numpy as np

percent_err=np.mean(np.abs((y_true-y_pred)/y_true))*100
print("Mean Absolute Percentage Error (MAPE) %:", percent_err)


Mean Absolute Percentage Error (MAPE) %: 12.112085890303183


Decision Trees and Random Forests

In [25]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error
import numpy as np

In [30]:
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_tr_scaled, y_tr_scaled)

y_dt_pred_scaled = dt_model.predict(X_ts_scaled)

# Inverse transform back to original target scale
y_dt_pred = scaler_y.inverse_transform(y_dt_pred_scaled.reshape(-1, 1))
y_ts_orig = scaler_y.inverse_transform(y_ts_scaled.reshape(-1, 1))

mae_dt = mean_absolute_error(y_ts_orig, y_dt_pred)
mse_dt = mean_squared_error(y_ts_orig, y_dt_pred)
mape_dt = np.mean(np.abs((y_ts_orig - y_dt_pred) / y_ts_orig)) * 100
acc_dt = 100 - mape_dt

print("Decision Tree Results (Original Units):")
print("MAE:", mae_dt)
print("MSE:", mse_dt)
print("MAPE %:", mape_dt)
print("Accuracy %:", acc_dt)

Decision Tree Results (Original Units):
MAE: 684.0653258497039
MSE: 2572891.123973599
MAPE %: 8.047244016357128
Accuracy %: 91.95275598364287


In [31]:
#Random Forest
rf_model=RandomForestRegressor(n_estimators=100,random_state=42)
rf_model.fit(X_tr_scaled,y_tr_scaled.ravel())

y_rf_pred_scaled=rf_model.predict(X_ts_scaled)

y_rf_pred=scaler_y.inverse_transform(y_rf_pred_scaled.reshape(-1,1))

In [32]:
mae_rf=mean_absolute_error(y_ts_orig,y_rf_pred)
mse_rf = mean_squared_error(y_ts_orig, y_rf_pred)
mape_rf = np.mean(np.abs((y_ts_orig - y_rf_pred) / y_ts_orig)) * 100
acc_rf = 100 - mape_rf

In [33]:

print("\nRandom Forest Results (Original Units):")
print("MAE:", mae_rf)
print("MSE:", mse_rf)
print("MAPE %:", mape_rf)
print("Accuracy %:", acc_rf)


Random Forest Results (Original Units):
MAE: 617.2638568792971
MSE: 1663923.7321868443
MAPE %: 7.197441024876808
Accuracy %: 92.80255897512319
